In [1]:
def get_input(day):
    with open('../inputs/input%d.txt' % day) as f:
        return f.read().strip()

In [ ]:
day = 1
input = get_input(day)

m = {'(': 1, ')': -1}

print(sum(m[x] for x in input))

s = 0
for i, x in enumerate(input):
    s += m[x]
    if s < 0:
        print(i+1)
        break

In [ ]:
day = 2
input = get_input(day)

s = 0
for line in input.split('\n'):
    [l, w, h] = map(int, line.split('x'))
    s += 2*l*w + 2*l*h + 2*w*h + min(l*w, l*h, w*h)
print(s)

s = 0
for line in input.split('\n'):
    [l, w, h] = map(int, line.split('x'))
    s += 2 * min(l+w, l+h, w+h) + l*w*h
print(s)

In [ ]:
day = 3
input = get_input(day)

m = {
    '^': lambda x,y: (x, y+1),
    'v': lambda x,y: (x, y-1),
    '>': lambda x,y: (x+1, y),
    '<': lambda x,y: (x-1, y),
}

p = (0,0)
s = {p}
for x in input:
    p = m[x](*p)
    s.add(p)
print(len(s))

p1 = (0,0)
p2 = (0,0)
s = {p1}
for x1,x2 in zip(input[::2], input[1::2]):
    p1 = m[x1](*p1)
    p2 = m[x2](*p2)
    s.add(p1)
    s.add(p2)
print(len(s))

In [2]:
day = 4
input = get_input(day)

import hashlib
from itertools import count

def f(s):
    for i in count(1):
        if hashlib.md5((input + str(i)).encode('utf-8')).hexdigest().startswith(s):
            return i

print(f("00000"))

print(f("000000"))

117946
3938038


In [ ]:
day = 5
input = get_input(day)

import re

def is_nice(s):
    return len(re.findall(r'[aeiou]', s)) > 2 \
           and re.search(r'(.)\1', s) \
           and not re.search(r'ab|cd|pq|xy', s)
        
print(sum(1 for x in input.split('\n') if is_nice(x)))

def is_nice2(s):
    return re.search(r'(..).*\1', s) \
           and re.search(r'(.).\1', s)
    
print(sum(1 for x in input.split('\n') if is_nice2(x)))

In [ ]:
day = 6
input = get_input(day)

import re

def run_command(grid, cmd_str, fun1, fun2, fun3):
    m = re.match(r'(toggle|turn on|turn off) (\d+),(\d+) through (\d+),(\d+)', line)
    cmd = m.group(1)
    x1,y1,x2,y2 = map(int, m.group(2, 3, 4, 5))
    f = fun1 if cmd == "toggle" else fun2 if cmd == "turn on" else fun3
    for x in range(x1, x2+1):
        for y in range(y1, y2+1):
            grid[x][y] = f(grid[x][y])

grid = [[0 for _ in range(1000)] for _ in range(1000)]
for line in input.split('\n'):
    run_command(grid, line,
                lambda x: x ^ 1,
                lambda x: 1,
                lambda x: 0)

print(sum(map(sum, grid)))

grid = [[0 for _ in range(1000)] for _ in range(1000)]
for line in input.split('\n'):
    run_command(grid, line,
                lambda x: x + 2,
                lambda x: x + 1,
                lambda x: max(0, x-1))

print(sum(map(sum, grid)))

In [ ]:
day = 7
input = get_input(day)

import re
from collections import deque

ops = {
    'NOT': '~',
    'AND': '&',
    'OR': '|',
    'LSHIFT': '<<',
    'RSHIFT': '>>',
}

def parse(line):
    line = re.sub(r'([a-z]+)', 'nodes["\\1"]', line)
    expr, var = line.split(' -> ')
    for op, val in ops.items():
        expr = expr.replace(op, val)
    return var, expr

def eval_instr(instr, variables):
    try:
        exec(instr, {}, variables)
        return True
    except KeyError:
        return False

def get_node_value(instrs, varname):
    variables = {'nodes': {}}
    while len(instrs) > 0:
        instr = instrs.popleft()
        if not eval_instr(instr, variables):
            instrs.append(instr)
    return variables['nodes'][varname]

instrs = deque('{} = {}'.format(*parse(line)) 
               for line in  input.split('\n'))
sol1 = get_node_value(instrs, 'a')
print(sol1)

instr_map = dict(parse(line)
                 for line in  input.split('\n'))
instr_map['nodes["b"]'] = sol1
instrs = deque('{} = {}'.format(*pair) 
               for pair in instr_map.items())
sol2 = get_node_value(instrs, 'a')
print(sol2)

In [ ]:
# Alternate (way better) Day 7
day = 7
input = get_input(day)

import re
from functools import partial, reduce

ops = {
    'NOT': '~',
    'AND': '&',
    'OR': '|',
    'LSHIFT': '<<',
    'RSHIFT': '>>',
}

def memoize(f):
    class memodict(dict):
        def __missing__(self, key):
            ret = self[key] = f(key)
            return ret 
    return memodict().__getitem__

def build_wires(input):
    wires = {}
    for line in input.split('\n'):
        rhs, w = re.match(r'(.+) -> (\S+)', line).group(1, 2)
        rhs = reduce(lambda x, y: x.replace(y[0], y[1]), ops.items(), rhs)
        rhs = re.sub(r'([a-z]+)', 'wires["\\1"]()', rhs)
        wires[w] = partial(memoize(eval), rhs)
    return wires

wires = build_wires(input)
ans1 = wires['a']()
print(ans1)

wires = build_wires(input)
wires['b'] = lambda: ans1
print(wires['a']())

In [ ]:
day = 8
input = get_input(day)

s = 0
for line in input.split('\n'):
    s += len(line)
    s -= len(eval(line))
# Part 1
print(s)

# Part 2
print(sum(2 + line.count('"') + line.count('\\')
          for line in input.split('\n')))

In [ ]:
day = 9
input = get_input(day)

import re
from collections import defaultdict
from itertools import permutations

m = defaultdict(dict)
for line in input.split('\n'):
    x = re.match(r'(\S+) to (\S+) = (\d+)', line)
    p1, p2 = x.group(1, 2)
    dist = int(x.group(3))
    m[p1][p2] = dist
    m[p2][p1] = dist

mindist = float('inf')
maxdist = 0
for path in permutations(m.keys()):
    dist = sum(m[a][b] for a, b in zip(path, path[1:]))
    mindist = min(mindist, dist)
    maxdist = max(maxdist, dist)

# Part 1
print(mindist)
# Part 2
print(maxdist)

In [ ]:
day = 10
input = get_input(day)

from itertools import groupby

def look_and_say(input):
    return ''.join(str(len(list(v))) + k for k, v in groupby(input))

p1 = input
for _ in range(40):
    p1 = look_and_say(p1)
print(len(p1))

p2 = input
for _ in range(50):
    p2 = look_and_say(p2)
print(len(p2))

In [ ]:
day = 11
input = get_input(day)

import re

def incr_str(s):
    c = 1
    i = len(s) - 1
    _s = ''
    while c > 0 and i >= 0:
        _s = chr(max(ord('a'), (ord(s[i])+1) % (ord('z')+1))) + _s
        c = 1 if _s[0] == 'a' else 0
        i -= 1
    _s = s[:i+1] + _s
    return _s

def has_reqs(s):
    x = 'abcdefghijklmnopqrstuvwxyz'
    return any(z in s for z in [''.join(y) for y in zip(x, x[1:], x[2:])]) \
           and 'i' not in s and 'o' not in s and 'l' not in s \
           and len(re.findall(r'(.)\1', s)) > 1

pw = incr_str(input)
while not has_reqs(pw):
    pw = incr_str(pw)
print(pw)

pw = incr_str(pw)
while not has_reqs(pw):
    pw = incr_str(pw)
print(pw)

In [ ]:
day = 12
input = get_input(day)

import json
import re

def find_ints(s):
    return map(int, re.findall('-?\d+', s))

print(sum(find_ints(input)))

def remove_red(obj):
    if "red" in obj.values():
        return {}
    return obj

print(sum(find_ints(str(json.loads(input, object_hook=remove_red)))))

In [ ]:
day = 13
input = get_input(day)

import re
from collections import defaultdict
from itertools import permutations

m = defaultdict(dict)
for line in input.split('\n'):
    x = re.match(r'(\S+) would (lose|gain) (\d+) happiness units by sitting next to (\S+)\.', line)
    p1, op, ch, p2 = x.group(1, 2, 3, 4)
    m[p1][p2] = -int(ch) if op == 'lose' else int(ch)

def find_max_happiness(m):
    maxhappiness = 0
    for ordering in permutations(m.keys()):
        happiness = sum(m[a][b]+m[b][a] for a, b in zip(ordering, ordering[1:]))
        happiness += m[ordering[0]][ordering[-1]] + m[ordering[-1]][ordering[0]]
        maxhappiness = max(maxhappiness, happiness)
    return maxhappiness

print(find_max_happiness(m))

for k in list(m.keys()):
    m['me'][k] = 0
    m[k]['me'] = 0

print(find_max_happiness(m))

In [ ]:
day = 14
input = get_input(day)

import re
from collections import Counter
from itertools import accumulate, repeat, cycle, islice, chain

class Reindeer(object):
    def __init__(self, name, speed, fly_time, rest_time):
        self.name = name
        self.speed = speed
        self.fly_time = fly_time
        self.rest_time = rest_time

total = 2503
m = {}

for line in input.split('\n'):
    x = re.match('(\S+) .* (\d+) .* (\d+) .* (\d+) .*\.', line)
    name, speed, t1, t2 = x.group(1, 2, 3, 4)
    speed, t1, t2 = int(speed), int(t1), int(t2)
    m[name] = Reindeer(name, speed, t1, t2)

dists = {}
for k, v in m.items():
    period = v.fly_time + v.rest_time
    dists[k] = v.speed * (v.fly_time * (total // period) + min(total % period, v.fly_time))
print(max(dists.values()))

dist_stages = {}
for k,v in m.items():
    speed_stages = cycle(chain(repeat(v.speed, v.fly_time), 
                               repeat(0, v.rest_time)))
    dist_stages[k] = islice(accumulate(speed_stages), total)

scores = [i for a in zip(*dist_stages.values()) 
          for i, v in enumerate(a) if v == max(a)]
print(max(Counter(scores).values()))


In [3]:
day = 15
input = get_input(day)

import re
import numpy as np

from functools import reduce
from operator import mul

m = []
for line in input.split('\n'):
    c, d, f, t, cal = map(int, re.findall('-?\d+', line))
    m.append([c, d, f, t, cal])
m = np.array(m)

def min_zero_sum(*ns):
    return max(0, sum(ns))

scores = [(reduce(mul, map(min_zero_sum, 
                           *map(mul, [i, j, k, l], m[:, :-1]))),
           sum(map(mul, [i, j, k, l], m[:, -1])))
          for i in range(101) 
          for j in range(0, 101-i) 
          for k in range(0, 101-j-i) 
          for l in [100 - i - j - k]]
# Part 1
print(max(s[0] for s in scores))

# Part 2
print(max(s[0] for s in scores if s[1] == 500))

21367368
1766400


In [4]:
# Alternate Day 15
day = 15
input = get_input(day)

import re
import numpy as np

from sympy import symbol
from scipy.optimize import minimize

from functools import reduce
from operator import mul

m = []
for line in input.split('\n'):
    c, d, f, t, cal = map(int, re.findall('-?\d+', line))
    m.append([c, d, f, t, cal])
m = np.array(m)

a, b, c, d = symbols('a b c d')
totals = [sum(x) for x in m[:,:-1].transpose()*np.array([a, b, c, d])]
expr = reduce(mul, totals)
def f(x):
    return -expr.subs({a: x[0], b: x[1], c: x[2], d: x[3]})
def rf(x):
    return expr.subs({a: x[0], b: x[1], c: x[2], d: x[3]})
ineqs = [lambda x: t.subs({a: x[0], b: x[1], c: x[2], d: x[3]}) for t in totals]
eqs = [lambda x: sum(x)-100]
cons = [{'type': 'eq', 'fun': x} for x in eqs] + [{'type': 'ineq', 'fun': x} for x in ineqs]
res = minimize(f, [25, 25, 25, 25], bounds=[(0,100), (0,100), (0,100), (0,100)], constraints=cons)
sol1 = rf([int(round(x)) for x in res.x])
print(sol1)
cons += [{'type': 'eq', 'fun': lambda x: sum(m[:,-1]*np.array([a, b, c, d])).subs({a: x[0], b: x[1], c: x[2], d: x[3]})-500}]
res = minimize(f, [0, 0, 100, 0], bounds=[(0,100), (0,100), (0,100), (0,100)], constraints=cons)
sol2 = rf([int(round(x)) for x in res.x])
print(sol2)


21367368
1766400


In [6]:
day = 16
input = get_input(day)

import re

m = []
pattern = r'Sue \d+: ([a-z]+): (\d+), ([a-z]+): (\d+), ([a-z]+): (\d+)'
for k1, a1, k2, a2, k3, a3 in re.findall(pattern, input):
    a1, a2, a3 = int(a1), int(a2), int(a3)
    m.append({k1: a1, k2: a2, k3: a3})

tape = {
    'children': 3,
    'cats': 7,
    'samoyeds': 2,
    'pomeranians': 3,
    'akitas': 0,
    'vizslas': 0,
    'goldfish': 5,
    'trees': 3,
    'cars': 2,
    'perfumes': 1,
}

def could_match(aunt):
    return all(tape[k] == v for k,v in aunt.items())

aunts = [i+1 for i,x in enumerate(m)
         if could_match(x)]
assert len(aunts) == 1
print(aunts[0])

def could_match2(aunt):
    for k,v in aunt.items():
        if k in ['cats', 'trees']:
            if tape[k] >= v:
                return False
        elif k in ['pomeranians', 'goldfish']:
            if tape[k] <= v:
                return False
        else:
            if tape[k] != v:
                return False
    return True

aunts = [i+1 for i,x in enumerate(m)
         if could_match2(x)]
assert len(aunts) == 1
print(aunts[0])

373
260


In [23]:
day = 17
input = get_input(day)

from itertools import combinations

m = [int(line) for line in input.split('\n')]

# Part 1 - Sum the number of all combinations that hold 150 L.
total = 0
for i in range(1, len(m)+1):
    total += len([x for x in combinations(m, i) if sum(x) == 150])
print(total)

# Part 2 - Break at the first combination that holds 150 L,
#    that's the smallest. Then just print the total as that's
#    the number of ways you can store 150 L in those containers.
total = 0
for i in range(1, len(m)+1):
    total += len([x for x in combinations(m, i) if sum(x) == 150])
    if total:
        break
print(total)

1304
18
